In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.



['salary_grid.csv', 'recruiting_costs.csv', 'HRDataset_v9.csv', 'production_staff.csv', 'core_dataset.csv']


In [2]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from time import time

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.linear_model import Lasso, Ridge


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb

In [9]:
hr = pd.read_csv("../input/core_dataset.csv")
hr

,Employee Name,Employee Number,State,Zip,DOB,Age,Sex,MaritalDesc,CitizenDesc,Hispanic/Latino,RaceDesc,Date of Hire,Date of Termination,Reason For Term,Employment Status,Department,Position,Pay Rate,Manager Name,Employee Source,Performance Score
0,"Brown, Mia",1.103024e+09,MA,1450.0,11/24/1985,32.0,Female,Married,US Citizen,No,Black or African American,10/27/2008,NaN,N/A - still employed,Active,Admin Offices,Accountant I,28.50,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
1,"LaRotonda, William",1.106027e+09,MA,1460.0,4/26/1984,33.0,Male,Divorced,US Citizen,No,Black or African American,1/6/2014,NaN,N/A - still employed,Active,Admin Offices,Accountant I,23.00,Brandon R. LeBlanc,Website Banner Ads,Fully Meets
2,"Steans, Tyrone",1.302053e+09,MA,2703.0,9/1/1986,31.0,Male,Single,US Citizen,No,White,9/29/2014,NaN,N/A - still employed,Active,Admin Offices,Accountant I,29.00,Brandon R. LeBlanc,Internet Search,Fully Meets
3,"Howard, Estelle",1.211051e+09,MA,2170.0,9/16/1985,32.0,Female,Married,US Citizen,No,White,2/16/2015,4/15/2015,N/A - still employed,Active,Admin Offices,Administrative Assistant,21.50,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review
4,"Singh, Nan",1.307060e+09,MA,2330.0,5/19/1988,29.0,Female,Single,US Citizen,No,White,5/1/2015,NaN,N/A - still employed,Active,Admin Offices,Administrative Assistant,16.56,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review
5,"Smith, Leigh Ann",7.110077e+08,MA,1844.0,6/14/1987,30.0,Female,Married,US Citizen,No,Asian,9/26/2011,9/25/2013,career change,Voluntarily Terminated,Admin Offices,Administrative Assistant,20.50,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
6,"LeBlanc, Brandon R",1.102024e+09,MA,1460.0,6/10/1984,33.0,Male,Married,US Citizen,No,White,1/5/2016,NaN,N/A - still employed,Active,Admin Offices,Shared Services Manager,55.00,Janet King,Monster.com,Fully Meets
7,"Quinn, Sean",1.206043e+09,MA,2045.0,11/6/1984,33.0,Male,Married,Eligible NonCitizen,No,Black or African American,2/21/2011,8/15/2015,career change,Voluntarily Terminated,Admin Offices,Shared Services Manager,55.00,Janet King,Diversity Job Fair,Fully Meets
8,"Boutwell, Bonalyn",1.307060e+09,MA,2468.0,4/4/1987,30.0,Female,Married,US Citizen,No,Asian,2/16/2015,NaN,N/A - still employed,Active,Admin Offices,Sr. Accountant,34.95,Brandon R. LeBlanc,Diversity Job Fair,90-day meets
9,"Foster-Baker, Amy",1.201031e+09,MA,2050.0,4/16/1979,38.0,Female,Married,US Citizen,no,White,1/5/2009,NaN,N/A - still employed,Active,Admin Offices,Sr. Accountant,34.95,Board of Directors,Other,Fully Meets


In [10]:
hr.columns = ['Name','EmpNum','State','Zip','Date_of_birth','Age','Sex','MaritalDesc','CitizenDesc','Hispanic/Latino','RaceDesc','Date_of_Hire','Date_of_Termination','Reason_For_Termination','Employment_Status','Department','Position','Pay_Rate','Manager_Name','Employment_Source','Performance_Score']
hr = hr.drop(hr.index[301])
print(hr)

                      Name            ...                    Performance_Score
0               Brown, Mia            ...                          Fully Meets
1     LaRotonda, William              ...                          Fully Meets
2         Steans, Tyrone              ...                          Fully Meets
3          Howard, Estelle            ...             N/A- too early to review
4              Singh, Nan             ...             N/A- too early to review
5         Smith, Leigh Ann            ...                          Fully Meets
6      LeBlanc, Brandon  R            ...                          Fully Meets
7              Quinn, Sean            ...                          Fully Meets
8        Boutwell, Bonalyn            ...                         90-day meets
9        Foster-Baker, Amy            ...                          Fully Meets
10             King, Janet            ...                          Fully Meets
11        Zamora, Jennifer            ...           

In [11]:
hr_no_dot = hr.drop('Date_of_Termination',axis=1)

In [12]:
le = preprocessing.LabelEncoder()
hr_no_dot = hr_no_dot.apply(le.fit_transform)
hr_no_dot

,Name,EmpNum,State,Zip,Date_of_birth,Age,Sex,MaritalDesc,CitizenDesc,Hispanic/Latino,RaceDesc,Date_of_Hire,Reason_For_Termination,Employment_Status,Department,Position,Pay_Rate,Manager_Name,Employment_Source,Performance_Score
0,28,82,10,3,51,7,0,1,2,0,2,17,2,0,0,0,35,3,3,3
1,149,102,10,4,143,8,1,0,2,0,2,8,2,0,0,0,22,3,20,3
2,262,182,10,119,266,6,1,3,2,0,5,88,2,0,0,0,38,3,7,3
3,126,168,10,86,274,7,0,1,2,0,5,27,2,0,0,1,19,3,15,4
4,249,200,10,98,164,4,0,3,2,0,5,46,2,0,0,1,6,3,20,4
5,253,4,10,25,198,5,0,1,2,0,1,86,4,4,0,1,16,3,3,3
6,155,76,10,4,194,8,1,1,2,0,5,7,2,0,0,18,73,9,9,3
7,221,158,10,47,58,8,1,1,0,0,2,31,4,4,0,18,73,9,3,3
8,24,206,10,112,148,5,0,1,2,0,1,27,2,0,0,21,43,3,3,0
9,88,138,10,49,133,13,0,1,2,2,5,4,2,0,0,21,43,2,13,3


In [13]:
corr = hr_no_dot.corr()
print(corr)

                            Name        ...          Performance_Score
Name                    1.000000        ...                   0.050567
EmpNum                 -0.062629        ...                   0.116753
State                  -0.068161        ...                   0.083885
Zip                     0.063956        ...                  -0.035418
Date_of_birth          -0.021010        ...                   0.058144
Age                     0.013301        ...                  -0.050695
Sex                     0.030142        ...                   0.020414
MaritalDesc            -0.043863        ...                   0.045112
CitizenDesc            -0.082492        ...                  -0.059613
Hispanic/Latino        -0.042185        ...                  -0.031689
RaceDesc               -0.058819        ...                  -0.045488
Date_of_Hire           -0.011273        ...                   0.173837
Reason_For_Termination  0.045477        ...                   0.007913
Employ

In [14]:
hr_no_dot.columns

Index(['Name', 'EmpNum', 'State', 'Zip', 'Date_of_birth', 'Age', 'Sex',
       'MaritalDesc', 'CitizenDesc', 'Hispanic/Latino', 'RaceDesc',
       'Date_of_Hire', 'Reason_For_Termination', 'Employment_Status',
       'Department', 'Position', 'Pay_Rate', 'Manager_Name',
       'Employment_Source', 'Performance_Score'],
      dtype='object')

In [15]:
X = hr_no_dot.loc[:,['State', 'Zip', 'Date_of_birth', 'Age', 'Sex',
       'MaritalDesc', 'CitizenDesc', 'Hispanic/Latino', 'RaceDesc',
       'Date_of_Hire', 'Reason_For_Termination', 'Employment_Status',
       'Department', 'Position', 'Manager_Name',
       'Employment_Source', 'Performance_Score']]
y = hr_no_dot.loc[:,['Pay_Rate']]

print(X)
print(y)

     State  Zip        ...          Employment_Source  Performance_Score
0       10    3        ...                          3                  3
1       10    4        ...                         20                  3
2       10  119        ...                          7                  3
3       10   86        ...                         15                  4
4       10   98        ...                         20                  4
5       10   25        ...                          3                  3
6       10    4        ...                          9                  3
7       10   47        ...                          3                  3
8       10  112        ...                          3                  0
9       10   49        ...                         13                  3
10      10   35        ...                         15                  3
11      10   53        ...                          4                  2
12      10   41        ...                         

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=17)

In [17]:
regressors = [
    LinearRegression(), 
    RANSACRegressor(), 
    KNeighborsRegressor(),
    KNeighborsRegressor(n_neighbors=9, metric='manhattan'),
    SVR(),
    LinearSVR(),
    GaussianProcessRegressor(),
    SVR(kernel='linear'), # Cf. LinearSVR: much slower, might be better or worse: 
]

In [18]:
head = 6
for model in regressors[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    predictions = model.predict(X_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs)" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Training time: 0.162s)
	Prediction time: 0.001s
	Explained variance: 0.2617233150129802
	Mean absolute error: 21.053629248570044
	R2 score: 0.2588572686093086

RANSACRegressor(base_estimator=None, is_data_valid=None, is_model_valid=None,
        loss='absolute_loss', max_skips=inf, max_trials=100,
        min_samples=None, random_state=None, residual_threshold=None,
        stop_n_inliers=inf, stop_probability=0.99, stop_score=inf)
	Training time: 0.105s)
	Prediction time: 0.001s
	Explained variance: -2.6448236155568234
	Mean absolute error: 42.36766136907169
	R2 score: -3.2812530949153738

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')
	Training time: 0.002s)
	Prediction time: 0.002s
	Explained variance: 0.23876125635750223
	Mean absolute error: 21.819354838709682
	R2 score: 0.2223

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

In [19]:
lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()

for model in [lr, lasso, ridge]:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Explained variance: 0.2617233150129802
	Mean absolute error: 21.053629248570044
	R2 score: 0.2588572686093086

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
	Explained variance: 0.2670900679656003
	Mean absolute error: 21.60418641465566
	R2 score: 0.26279836839698656

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
	Explained variance: 0.26235856258562507
	Mean absolute error: 21.055164798726356
	R2 score: 0.2595085723662669



In [20]:
regressors = [LinearRegression(),
              DecisionTreeRegressor(max_depth=5),
              DecisionTreeRegressor(max_depth=10),
              DecisionTreeRegressor(max_depth=20),
              RandomForestRegressor(max_depth=10),
              SVR(),]

for model in regressors:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Explained variance: 0.2617233150129802
	Mean absolute error: 21.053629248570044
	R2 score: 0.2588572686093086

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
	Explained variance: 0.8395094363040299
	Mean absolute error: 7.9409531894251755
	R2 score: 0.8368040970971553

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
	Explained variance: 0.7614763397099285
	Mean absolute error: 10.2

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
 